In [4]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor

### 从datasets下载数据，并完成数据划分

In [2]:
# 训练集
training_data = datasets.FashionMNIST(root='data', train=True, transform=ToTensor(), download=True)
# 测试集
test_data = datasets.FashionMNIST(root='data', train=False, transform=ToTensor(), download=True)

100%|██████████| 26.4M/26.4M [00:03<00:00, 8.71MB/s]


Extracting data\FashionMNIST\raw\train-images-idx3-ubyte.gz to data\FashionMNIST\raw



100%|██████████| 29.5k/29.5k [00:00<00:00, 187kB/s]


Extracting data\FashionMNIST\raw\train-labels-idx1-ubyte.gz to data\FashionMNIST\raw



100%|██████████| 4.42M/4.42M [00:01<00:00, 3.43MB/s]


Extracting data\FashionMNIST\raw\t10k-images-idx3-ubyte.gz to data\FashionMNIST\raw



100%|██████████| 5.15k/5.15k [00:00<?, ?B/s]

Extracting data\FashionMNIST\raw\t10k-labels-idx1-ubyte.gz to data\FashionMNIST\raw



### 将 Dataset 作为参数传递给 DataLoader。这将包装数据集上的一个可迭代对象，并支持自动批处理、采样、洗牌和多进程数据加载。这里我们定义一个批次大小为 64，即数据加载器可迭代对象中的每个元素将返回一个包含 64 个特征和标签的批次。

In [5]:
batch_size = 64

# Create data loaders.
train_dataloader = DataLoader(training_data, batch_size=batch_size)
test_dataloader = DataLoader(test_data, batch_size=batch_size)

for X, y in test_dataloader:
    print(f"Shape of X [N, C, H, W]: {X.shape}")
    print(f"Shape of y: {y.shape} {y.dtype}")
    break

Shape of X [N, C, H, W]: torch.Size([64, 1, 28, 28])
Shape of y: torch.Size([64]) torch.int64


## 创建模型
为了在 PyTorch 中定义一个神经网络，我们创建一个继承自 nn.Module 的类。我们在 __init__ 函数中定义网络的层，并在 forward 函数中指定数据如何通过网络传递。为了加速神经网络中的操作，我们将它移动到 GPU 或 MPS（如果可用）

In [6]:
# 选择训练所用硬件
device = (
    "cuda"
    if torch.cuda.is_available()
    else "mps"
    if torch.backends.mps.is_available()
    else "cpu"
)
print(f"Using {device} device")

# 定义模型
class NeuralNetwork(nn.Module):
    def __init__(self):
        super().__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10)
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

model = NeuralNetwork().to(device)
print(model)

Using cuda device
NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


# 优化模型参数
为了训练模型，我们需要一个**损失函数**和一个**优化器**。

In [7]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=1e-3)

在一个训练循环中，模型对训练数据集（以批次形式馈送到它）进行预测，并将预测误差反向传播以调整模型的参数。

In [8]:
def train(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    model.train()
    for batch, (X, y) in enumerate(dataloader):
        X, y = X.to(device), y.to(device)

        # Compute prediction error
        pred = model(X)
        loss = loss_fn(pred, y)

        # Backpropagation
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

        if batch % 100 == 0:
            loss, current = loss.item(), (batch + 1) * len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")

我们还会检查模型对测试数据集的性能，以确保它正在学习。

In [9]:
def test(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    model.eval()
    test_loss, correct = 0, 0
    with torch.no_grad():
        for X, y in dataloader:
            X, y = X.to(device), y.to(device)
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()
    test_loss /= num_batches
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

训练过程在几个迭代（epochs）中进行。在每个 epoch 期间，模型学习参数以做出更好的预测。我们在每个 epoch 打印模型的准确性和损失；我们希望看到准确性随着每个 epoch 的增加而增加，而损失随着每个 epoch 的增加而降低。

In [10]:
epochs = 5
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train(train_dataloader, model, loss_fn, optimizer)
    test(test_dataloader, model, loss_fn)
print("Done!")

Epoch 1
-------------------------------
loss: 2.302278  [   64/60000]
loss: 2.290190  [ 6464/60000]
loss: 2.270245  [12864/60000]
loss: 2.271246  [19264/60000]
loss: 2.253623  [25664/60000]
loss: 2.221846  [32064/60000]
loss: 2.234025  [38464/60000]
loss: 2.197290  [44864/60000]
loss: 2.192871  [51264/60000]
loss: 2.169053  [57664/60000]
Test Error: 
 Accuracy: 44.2%, Avg loss: 2.163829 

Epoch 2
-------------------------------
loss: 2.170492  [   64/60000]
loss: 2.164888  [ 6464/60000]
loss: 2.107510  [12864/60000]
loss: 2.128720  [19264/60000]
loss: 2.075923  [25664/60000]
loss: 2.019458  [32064/60000]
loss: 2.052581  [38464/60000]
loss: 1.973479  [44864/60000]
loss: 1.977960  [51264/60000]
loss: 1.917539  [57664/60000]
Test Error: 
 Accuracy: 55.4%, Avg loss: 1.911203 

Epoch 3
-------------------------------
loss: 1.942814  [   64/60000]
loss: 1.918433  [ 6464/60000]
loss: 1.801076  [12864/60000]
loss: 1.843257  [19264/60000]
loss: 1.730291  [25664/60000]
loss: 1.685104  [32064/600

# 保存模型
保存模型的一种常见方法是序列化内部状态字典（包含模型参数）。

In [11]:
torch.save(model.state_dict(), "model.pth")
print("Saved PyTorch Model State to model.pth")

Saved PyTorch Model State to model.pth


# 加载模型
加载模型的过程包括重新创建模型结构并将状态字典加载到其中。

In [12]:
model = NeuralNetwork().to(device)
model.load_state_dict(torch.load("model.pth", weights_only=True))

<All keys matched successfully>

# 该模型已经可以进行预测

In [13]:
classes = [
    "T-shirt/top",
    "Trouser",
    "Pullover",
    "Dress",
    "Coat",
    "Sandal",
    "Shirt",
    "Sneaker",
    "Bag",
    "Ankle boot",
]

model.eval()
x, y = test_data[0][0], test_data[0][1]
with torch.no_grad():
    x = x.to(device)
    pred = model(x)
    predicted, actual = classes[pred[0].argmax(0)], classes[y]
    print(f'Predicted: "{predicted}", Actual: "{actual}"')

Predicted: "Ankle boot", Actual: "Ankle boot"
